In [1]:
#用于1车道1交通灯和 pytorch测试
#作者lukeliuli@163.com
# -*- coding: utf-8 -*-
"""
@author: lukeliuli@163.com
"""
#%cd /root/myPrjs/SUMONBDT
import torch
import torchvision
import torchvision.transforms as transforms

import torchvision.models as models
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim

import time
import numpy as np
import os
import sys

import random
import csv
import datetime


def sortTakeSecond(elem):
    return elem[1] 



os.environ['SUMO_HOME'] = '/home/liuli/myPrjs/sumo/'
if 'SUMO_HOME' in os.environ:
     tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
     sys.path.append(tools)
else:
     sys.exit("please declare environment variable 'SUMO_HOME'")
    


import libsumo

path = os.getcwd()
path2 = path+"\sumoCfgs\my1Lane1TlsVeh3-server.sumocfg"#window
path2 = path+"/sumoCfgs1/my1Lane1TlsVeh3-server.sumocfg"#linux
sumoBinary = "sumo"
libsumo.start([sumoBinary, "-c", path2])


#定义模拟过程中动态可调整参数
params ={}
params["GreenTime"] = 15 #红灯时间
params["redTime"] = 25   #绿灯时间
params["ObjVehPosInXY"] = [-150,0] #主车位置
params["MaxAcc"] = 1              #最大加速度
params["DrvResTime"] = 0.5     # 驾驶员反应时间
params["addedVeh"] = 2+1        #停车线前面有多少车
params["accel"]=2.8     #最大加速度
params["tau"]=1  #驾驶员最大反应时间
    
numSim = 1000
nextTLSID = "2"
veh = "00003"
counterNoLowSpeed = 0
dataAllSim = []
for ii in range(numSim):
    print("###################################################################start sim %d" %ii)
    str1 = "./trainData/redTLS_"+str(ii)+"_sim.csv"
    csv_file = open(str1,"w+")    
    writer = csv.writer(csv_file)   



    libsumo.start([sumoBinary, "-c", path2])
    stepNum = 0
    requireStop = 0
    redLowSpeed = 0
    data = []
    ##修改出发位置，随机参数
    X =  params["ObjVehPosInXY"][0]+min(random.random()*150,80)#注意坐标的原点
    Y =  params["ObjVehPosInXY"][1]
    libsumo.vehicle.moveToXY(veh,"e1to2", 0, X, Y, -1073741824.0, 1)
    
    ##修改加速，反应时间
    vehOthers = "EBUS2"
    accel = params["accel"]
    accel = max(accel*random.random(),1)
    libsumo.vehicletype.setAccel(vehOthers,accel)
    
    tau = params["tau"]
    tau = max(tau*random.random(),0.3)
    libsumo.vehicletype.setTau(vehOthers,tau)
    
    ####修改绿灯时间，随机参数
    """
    greenTime =  params["GreenTime"]#注意坐标的原点
    greenTime = max(random.random()*greenTime,0.1)
    allProgramLogics = libsumo.trafficlight.getAllProgramLogics(nextTLSID)
    lgc1 = allProgramLogics[0]
    yellowDurTime = lgc1.phases[0].duration
    greenDurTime = lgc1.phases[1].duration
    yellowDurTime = lgc1.phases[2].duration
    redDurTime = lgc1.phases[3].duration
    
    lgc1.phases[1].duration = greenTime
    libsumo.trafficlight.setProgramLogic(nextTLSID,lgc1) 
    phaseDur = libsumo.trafficlight.getPhaseDuration(nextTLSID)
    """
    
    ####修改逻辑和红灯时间，随机参数
    redTime =  params["redTime"]#注意坐标的原点
    redTime = max(random.random()*redTime,0.1)
    allProgramLogics = libsumo.trafficlight.getAllProgramLogics(nextTLSID)
    lgc1 = allProgramLogics[0]
    yellowDurTime = lgc1.phases[0].duration
    
    lgc1.phases[1].state = 'r'
    redDurTime = lgc1.phases[1].duration
    
    yellowDurTime = lgc1.phases[2].duration
    
    lgc1.phases[3].state = 'G'
    greenDurTime = lgc1.phases[3].duration
    
    lgc1.phases[1].duration = redTime
    libsumo.trafficlight.setProgramLogic(nextTLSID,lgc1) 
    phaseDur = libsumo.trafficlight.getPhaseDuration(nextTLSID)
   
    #print(X,greenTime,greenDurTime,yellowDurTime,redDurTime)
    #print(lgc1)
    
    data = []
    
    ##先运行一步
    libsumo.simulationStep()
    stepNum += 1
    while requireStop == 0:
        libsumo.simulationStep()
        stepNum += 1
        
        ###交通灯设置
        ####https://sumo.dlr.de/daily/pydoc/traci._vehicle.html
        nextTLSID = "2"
        TLSPos = 0
        states = libsumo.trafficlight.getRedYellowGreenState(nextTLSID)
        timeT = libsumo.simulation.getCurrentTime()  # 当前时间
        nextSwitch = libsumo.trafficlight.getNextSwitch(nextTLSID)
        phaseName = libsumo.trafficlight.getPhaseName(nextTLSID)
        phase = libsumo.trafficlight.getPhase(nextTLSID)
        phaseDur = libsumo.trafficlight.getPhaseDuration(nextTLSID)
        curTime = timeT/1000
        phaseLeftTime = nextSwitch - curTime
        
      
         

        #车设置
        veh = "00003"
        vehPos = libsumo.vehicle.getPosition(veh)[0]
        speed = libsumo.vehicle.getSpeed(veh)
        vehMaxSpeed = libsumo.vehicle.getMaxSpeed(veh)

        laneID = libsumo.vehicle.getLaneID(veh)
        #meanSpeed = libsumo.lane.getLastStepMeanSpeed(laneID)
        laneMaxSpeed = libsumo.lane.getMaxSpeed(laneID)

        vehMaxSpeed = min(vehMaxSpeed, laneMaxSpeed)
        
        dist = TLSPos-vehPos
        arrivalTime1 = min(100,dist / (speed + 0.001))
        arrivalTime2 = min(100,dist / vehMaxSpeed)
        #arrivalTime3 = min(100,dist / (meanSpeed+0.001))

        #predictMeanSpeed1min = laneMaxSpeed  #1分钟前绿灯时,当前场景下目标车道的平均速度
        #arrivalTime4 = min(dist / predictMeanSpeed1min,100)
        
        ##

       
        
        if states == "r" and  speed<5/3.6:
            print("low speed")
            redLowSpeed = 1
            requireStop = 1
            
        if states == "G" and  speed<5/3.6:
            print("low speed")
            redLowSpeed= 1
            requireStop = 1

      
            
            
        if timeT/1000 > 30 or dist<=0:  # if time is over 30 second stoping the simulation
            requireStop = 1
        #name1 = ["当前红灯剩余时间","距离红灯的位置","当前速度","车道预计平均速度","预计到达时间1","预计到达时间2"]    
        laneAVGSpeed = 40/3.6
        data1 =[phaseLeftTime,dist,speed,40/3.6,dist/(speed+0.01),dist/laneAVGSpeed]
        
        #前面只有停止的车，不存在运动的车,没有的车位置为0
        #vehTmp = "00001"
        #vehPos1 = libsumo.vehicle.getPosition(vehTmp)[0]
        #vehTmp = "00002"
        #vehPos2 = libsumo.vehicle.getPosition(vehTmp)[0]
        #name2 = ["红灯前车辆1位置","车辆2位置","车辆3位置","车辆4位置","车辆5位置","车辆6位置","车辆7位置","车辆8位置","速度一直高于1m/s"]    
        #laneAVGSpeed = 60/3.6
        #data2 =[TLSPos-vehPos1,TLSPos-vehPos2,0,0,0,0,0,0]
        #print("#####tls state: %s,duration:%.3f,running time(ms):%d,passTLS:%d,dist:%d,speed:%.2f" %(states,phaseDur,timeT,passTLSNoDeAcc,dist,speed))
        #data.append(data1+data2)
        
        
        ##获得车灯前的车.注意为了简单.最大为8辆车
        name2 = ["红灯前车辆1位置","红灯前车辆1速度","车辆2位置","车辆2速度","车辆3位置","车辆3速度","车辆4位置","车辆4速度"]
        name3 = ["红灯前车辆5位置","红灯前车辆5速度","车辆6位置","车辆6速度","车辆7位置","车辆7速度","车辆8位置","车辆8速度"]
       
        data2 =[0] * 16;
        vehicles = libsumo.vehicle.getIDList();  # 当前在道路上的车辆
        vehListTmp = []
        for vehTmp in vehicles:
            tlsList = libsumo.vehicle.getNextTLS(vehTmp)
            if vehTmp == "00003" or len(tlsList) == 0:
                continue
            tlsList = libsumo.vehicle.getNextTLS(vehTmp)
            tlsID_T, tlsIndex, distTmp, state = tlsList[0]
            speed = libsumo.vehicle.getSpeed(vehTmp)
            
            if tlsID_T == "2":
                vehListTmp.append([vehTmp,distTmp,speed]) 
                
        if len(vehListTmp)>0:
            vehListTmp.sort(key=sortTakeSecond)
            for i in range(len(vehListTmp)):
                data2[2*i]   = vehListTmp[i][1]
                data2[2*i+1] = vehListTmp[i][2]
                
        data.append(data1+data2)
                
             

                
       
    
    
    #####################################速度分段标记<10为0.10<= <20 为1 《=-20 《30 为2。30《 为3 
    speedFlag = -1
    veh = "00003"
    if redLowSpeed == 1:
        speedFlag = 0
        
    if redLowSpeed == 0:
        speed = libsumo.vehicle.getSpeed(veh)
        print("########################################speed:",speed)
        if speed>=10/3.6 and speed <20/3.6:
            speedFlag = 1
        if speed>=20/3.6 and speed <30/3.6:
            speedFlag = 2
        if speed>=30/3.6:
            speedFlag = 3
        print("############speedFlag:",speedFlag)
        print("############speedFlag:",speedFlag)
        
    
    hspeedFlag = speedFlag #for 层次型速度标志 
    hstruct = [2,3,5,9]
    if redLowSpeed == 1:
        hspeedFlag = [0,0,0,0];
        
    if redLowSpeed == 0:
        speed = libsumo.vehicle.getSpeed(veh)
        print("########################################speed:",speed)
        hstruct = [2,3,5,9]
        if speed <5/3.6:
            hspeedFlag = [0,0,0,0]
        if speed>=5/3.6 and speed <10/3.6:
            hspeedFlag = [0,0,0,1]
        if speed>=10/3.6 and speed <15/3.6:
            hspeedFlag = [1,1,1,2]
        if speed>=15/3.6 and speed <20/3.6:
            hspeedFlag = [1,1,1,3]
        if speed>=20/3.6 and speed <25/3.6:
            hspeedFlag = [1,1,2,4]
        if speed>=25/3.6 and speed <30/3.6:
            hspeedFlag = [1,1,2,5]
        if speed>=30/3.6 and speed <35/3.6:
            hspeedFlag = [1,2,3,6]
        if speed>=35/3.6 and speed <40/3.6:
            hspeedFlag = [1,2,3,7]
        if speed>=40/3.6:
            hspeedFlag = [1,2,4,8]

        print("############HspeedFlag:",hspeedFlag)
        print("############HspeedFlag:",hspeedFlag)

    
    ########################################
    time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    libsumo.close()
    print("###########################################################%s, sim %d close" %(time,ii))
    name1 = ["当前红灯剩余时间","距离红灯的位置","当前速度","车道预计平均速度","预计到达时间1","预计到达时间2"]   
    name2 = ["红灯前车辆1位置","红灯前车辆1速度","车辆2位置","车辆2速度","车辆3位置","车辆3速度","车辆4位置","车辆4速度"]
    name3 = ["红灯前车辆5位置","红灯前车辆5速度","车辆6位置","车辆6速度","车辆7位置","车辆7速度","车辆8位置","车辆8速度"]
    name4 = ["H0","H1","H2","H3"]   
    nameTmp = name1+name2+name3+name4
    writer.writerow(nameTmp)
    #print(len(data))
    for i in range(len(data)):
        rowTmp=data[i]
        #print(type(rowTmp))
        #rowTmp.extend([speedFlag])
        rowTmp.extend([hspeedFlag[0]])
        rowTmp.extend([hspeedFlag[1]])
        rowTmp.extend([hspeedFlag[2]])
        rowTmp.extend([hspeedFlag[3]])
        #print(rowTmp)
        writer.writerow(rowTmp)
        dataAllSim.append(rowTmp)
    csv_file.close()
    
    
   
    
    if redLowSpeed == 0:
        counterNoLowSpeed= counterNoLowSpeed+1
        
print("finish numSim:%d,LowSpeed:%d" %(numSim,numSim-counterNoLowSpeed))

#########################################################################################################
name1 = ["当前红灯剩余时间","距离红灯的位置","当前速度","车道预计平均速度","预计到达时间1","预计到达时间2"]   
name2 = ["红灯前车辆1位置","红灯前车辆1速度","车辆2位置","车辆2速度","车辆3位置","车辆3速度","车辆4位置","车辆4速度"]
name3 = ["红灯前车辆5位置","红灯前车辆5速度","车辆6位置","车辆6速度","车辆7位置","车辆7速度","车辆8位置","车辆8速度"]
       
nameTmp = name1+name2+name3+name4
name1 = ["redLightTime","distToRedLight","speed","laneAvgSpeed","arriveTime1","arriveTime2"]   
name2 = ["vehPos_1","vehSpeed_1","vehPos_2","vehSpeed_2","vehPos_3","vehSpeed_3","vehPos_4","vehSpeed_4"] 
name3 = ["vehPos_5","vehSpeed_5","vehPos_6","vehSpeed_6","vehPos_7","vehSpeed_7","vehPos_8","vehSpeed_8"] 
headers =  name1+name2+name3+name4
str1 = "./trainData/dataAllSim.csv"
with open(str1,'w')as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(dataAllSim)
#############################################################################################################    
!rm -rf ./trainData/redTLS*.csv
!zip -q  ./trainData/dataAllSim.zip ./trainData/dataAllSim.csv
#!rm -rf ./trainData/*.csv

###################################################################start sim 0
########################################speed: 6.379130636019545
############speedFlag: 2
############speedFlag: 2
########################################speed: 6.379130636019545
############HspeedFlag: [1, 1, 2, 4]
############HspeedFlag: [1, 1, 2, 4]
###########################################################2022-05-18 08:25:17, sim 0 close
###################################################################start sim 1
########################################speed: 5.348501254169431
############speedFlag: 1
############speedFlag: 1
########################################speed: 5.348501254169431
############HspeedFlag: [1, 1, 1, 3]
############HspeedFlag: [1, 1, 1, 3]
###########################################################2022-05-18 08:25:17, sim 1 close
###################################################################start sim 2
low speed
###########################################################20

low speed
###########################################################2022-05-18 08:25:19, sim 34 close
###################################################################start sim 35
low speed
###########################################################2022-05-18 08:25:19, sim 35 close
###################################################################start sim 36
low speed
###########################################################2022-05-18 08:25:19, sim 36 close
###################################################################start sim 37
########################################speed: 5.850552199935778
############speedFlag: 2
############speedFlag: 2
########################################speed: 5.850552199935778
############HspeedFlag: [1, 1, 2, 4]
############HspeedFlag: [1, 1, 2, 4]
###########################################################2022-05-18 08:25:19, sim 37 close
###################################################################start sim 38
low speed
##############

########################################speed: 4.335037325779387
############speedFlag: 1
############speedFlag: 1
########################################speed: 4.335037325779387
############HspeedFlag: [1, 1, 1, 3]
############HspeedFlag: [1, 1, 1, 3]
###########################################################2022-05-18 08:25:21, sim 68 close
###################################################################start sim 69
########################################speed: 6.132268918516874
############speedFlag: 2
############speedFlag: 2
########################################speed: 6.132268918516874
############HspeedFlag: [1, 1, 2, 4]
############HspeedFlag: [1, 1, 2, 4]
###########################################################2022-05-18 08:25:21, sim 69 close
###################################################################start sim 70
low speed
###########################################################2022-05-18 08:25:22, sim 70 close
###########################################

low speed
###########################################################2022-05-18 08:25:24, sim 104 close
###################################################################start sim 105
low speed
###########################################################2022-05-18 08:25:24, sim 105 close
###################################################################start sim 106
low speed
###########################################################2022-05-18 08:25:24, sim 106 close
###################################################################start sim 107
low speed
###########################################################2022-05-18 08:25:24, sim 107 close
###################################################################start sim 108
low speed
###########################################################2022-05-18 08:25:24, sim 108 close
###################################################################start sim 109
low speed
###########################################################2022-0

########################################speed: 4.0086866216637995
############speedFlag: 1
############speedFlag: 1
########################################speed: 4.0086866216637995
############HspeedFlag: [1, 1, 1, 2]
############HspeedFlag: [1, 1, 1, 2]
###########################################################2022-05-18 08:25:26, sim 138 close
###################################################################start sim 139
low speed
###########################################################2022-05-18 08:25:26, sim 139 close
###################################################################start sim 140
low speed
###########################################################2022-05-18 08:25:26, sim 140 close
###################################################################start sim 141
low speed
###########################################################2022-05-18 08:25:26, sim 141 close
###################################################################start sim 142
low speed
####

...

low speed
###########################################################2022-05-18 08:26:14, sim 862 close
###################################################################start sim 863
low speed
###########################################################2022-05-18 08:26:14, sim 863 close
###################################################################start sim 864
low speed
###########################################################2022-05-18 08:26:14, sim 864 close
###################################################################start sim 865
########################################speed: 4.43387016878657
############speedFlag: 1
############speedFlag: 1
########################################speed: 4.43387016878657
############HspeedFlag: [1, 1, 1, 3]
############HspeedFlag: [1, 1, 1, 3]
###########################################################2022-05-18 08:26:14, sim 865 close
###################################################################start sim 866
##################

low speed
###########################################################2022-05-18 08:26:16, sim 896 close
###################################################################start sim 897
low speed
###########################################################2022-05-18 08:26:16, sim 897 close
###################################################################start sim 898
low speed
###########################################################2022-05-18 08:26:16, sim 898 close
###################################################################start sim 899
low speed
###########################################################2022-05-18 08:26:16, sim 899 close
###################################################################start sim 900
########################################speed: 4.0897955014847955
############speedFlag: 1
############speedFlag: 1
########################################speed: 4.0897955014847955
############HspeedFlag: [1, 1, 1, 2]
############HspeedFlag: [1, 1, 1, 2]
####

########################################speed: 4.003264456466475
############speedFlag: 1
############speedFlag: 1
########################################speed: 4.003264456466475
############HspeedFlag: [1, 1, 1, 2]
############HspeedFlag: [1, 1, 1, 2]
###########################################################2022-05-18 08:26:18, sim 930 close
###################################################################start sim 931
########################################speed: 4.354748586626197
############speedFlag: 1
############speedFlag: 1
########################################speed: 4.354748586626197
############HspeedFlag: [1, 1, 1, 3]
############HspeedFlag: [1, 1, 1, 3]
###########################################################2022-05-18 08:26:18, sim 931 close
###################################################################start sim 932
low speed
###########################################################2022-05-18 08:26:18, sim 932 close
######################################

low speed
###########################################################2022-05-18 08:26:20, sim 960 close
###################################################################start sim 961
low speed
###########################################################2022-05-18 08:26:20, sim 961 close
###################################################################start sim 962
low speed
###########################################################2022-05-18 08:26:20, sim 962 close
###################################################################start sim 963
low speed
###########################################################2022-05-18 08:26:20, sim 963 close
###################################################################start sim 964
low speed
###########################################################2022-05-18 08:26:20, sim 964 close
###################################################################start sim 965
low speed
###########################################################2022-0

########################################speed: 4.101873980307328
############speedFlag: 1
############speedFlag: 1
########################################speed: 4.101873980307328
############HspeedFlag: [1, 1, 1, 2]
############HspeedFlag: [1, 1, 1, 2]
###########################################################2022-05-18 08:26:22, sim 996 close
###################################################################start sim 997
low speed
###########################################################2022-05-18 08:26:22, sim 997 close
###################################################################start sim 998
low speed
###########################################################2022-05-18 08:26:22, sim 998 close
###################################################################start sim 999
low speed
###########################################################2022-05-18 08:26:23, sim 999 close
finish numSim:1000,LowSpeed:742


In [ ]:
!rm -rf ./trainData/*.csv

In [ ]:
!git config --global user.email "lukeliuli@163.com"
!git config --global user.name "lukeliuli"
!git add .
!git commit -m  "tls OK, collectiong all and adding speedFlag"
!git push

In [5]:
!export SUMO_HOME="/home/liuli/myPrjs/sumo/"
!export PATH=$PATH:/home/liuli/myPrjs/sumo/bin